<a href="https://colab.research.google.com/github/Neelavo/Hackathons/blob/main/blackout_hackathon_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
traincsv=pd.read_csv("/content/Train_data_Autism.csv")

# Preprocessing

In [ ]:
traincsv.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'ethnicity', 'jaundice', 'austim', 'contry_of_res', 'used_app_before',
       'result', 'age_desc', 'relation', 'Class/ASD'],
      dtype='object')

In [ ]:
traincsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3697 entries, 0 to 3696
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   A1_Score         3697 non-null   object
 1   A2_Score         3697 non-null   object
 2   A3_Score         3697 non-null   object
 3   A4_Score         3697 non-null   object
 4   A5_Score         3697 non-null   object
 5   A6_Score         3697 non-null   object
 6   A7_Score         3697 non-null   object
 7   A8_Score         3697 non-null   object
 8   A9_Score         3697 non-null   object
 9   A10_Score        3697 non-null   object
 10  age              3695 non-null   object
 11  gender           3697 non-null   object
 12  ethnicity        3697 non-null   object
 13  jaundice         3697 non-null   object
 14  austim           3697 non-null   object
 15  contry_of_res    3697 non-null   object
 16  used_app_before  3697 non-null   object
 17  result           3697 non-null   

In [ ]:
traincsv=traincsv.drop(1196) #this guy just entered the column names for all of his answers :( (probably autistic)

In [ ]:
for col in traincsv.columns:
  if col not in ('age', 'result'):
    print(f'{col}: {traincsv[col].unique()}')

A1_Score: ['1' '0']
A2_Score: ['0' '1']
A3_Score: ['1' '0']
A4_Score: ['0' '1']
A5_Score: ['1' '0']
A6_Score: ['0' '1']
A7_Score: ['1' '0']
A8_Score: ['0' '1']
A9_Score: ['1' '0']
A10_Score: ['1' '0']
gender: ['f' 'm']
ethnicity: ['?' 'White-European' 'Middle Eastern ' 'Pasifika' 'Black' 'Others'
 'Hispanic' 'Asian' 'Turkish' 'South Asian' 'Latino' 'others'
 "'Middle Eastern '" "'South Asian'"]
jaundice: ['no' 'yes']
austim: ['no' 'yes']
contry_of_res: ['Austria' 'India' 'United States' 'South Africa' 'Jordan'
 'United Kingdom' 'Brazil' 'New Zealand' 'Canada' 'Kazakhstan'
 'United Arab Emirates' 'Australia' 'Ukraine' 'Iraq' 'France' 'Malaysia'
 'Viet Nam' 'Egypt' 'Netherlands' 'Afghanistan' 'Oman' 'Italy'
 'AmericanSamoa' 'Bahamas' 'Saudi Arabia' 'Ireland' 'Aruba' 'Sri Lanka'
 'Russia' 'Bolivia' 'Azerbaijan' 'Armenia' 'Serbia' 'Ethiopia' 'Sweden'
 'Iceland' 'Hong Kong' 'Angola' 'China' 'Germany' 'Spain' 'Tonga'
 'Pakistan' 'Iran' 'Argentina' 'Japan' 'Mexico' 'Nicaragua' 'Sierra Leone'


In [ ]:
def fill_na_with_column_average(df, column_name):
    average = df[column_name].dropna().mean()
    df[column_name] = df[column_name].fillna(average)
    num_nan_after_replacement = df[column_name].isna().sum()
    print(num_nan_after_replacement)
    return df

In [ ]:
def fill_null_values_with_probabilities(df, column_name, replacement_strings, replacement_probabilities):
    num_null_values = df[column_name].isnull().sum()
    replacement_distribution = np.random.choice(replacement_strings, size=num_null_values, p=replacement_probabilities)
    df[column_name].fillna(pd.Series(replacement_distribution, index=df.index[df[column_name].isnull()]), inplace=True)
    return df

In [ ]:
traincsv['jaundice'] = traincsv['jaundice'].replace({"no":0, "yes":1})
traincsv['gender'] = traincsv['gender'].replace({"m":1, "f":0})
traincsv['used_app_before'] = traincsv['used_app_before'].replace({'no':0,"yes":1})
traincsv['relation'] = traincsv['relation'].replace({"Others":"others","Self":"self", "'Health care professional'":"Health care professional", "?":"others"})
traincsv['ethnicity']=traincsv['ethnicity'].replace({"Others": "others", "?":"others", "'Middle Eastern '":"Middle Eastern ", "Middle Eastern":"Middle Eastern ", "'South Asian'":"South Asian"})
traincsv['age_desc'] = traincsv['age_desc'].replace({"'12-15 years'":"'12-16 years'", "18 and more":"'18 and more'"})
traincsv['austim']=traincsv['austim'].replace({'yes':1, 'no':0})
traincsv['age']=traincsv['age'].replace({'?':None})
traincsv['age']=traincsv['age'].astype(np.float64)
traincsv=fill_na_with_column_average(traincsv, 'age')

for i in range(10):
  traincsv[f'A{i+1}_Score']=traincsv[f'A{i+1}_Score'].apply(lambda x: int(x))
results=traincsv[['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']].sum(axis=1)
traincsv['result']=results

traincsv['ethnicity'] = traincsv['ethnicity'].replace('others', None)
traincsv['relation'] = traincsv['relation'].replace('others', None)
traincsv = fill_null_values_with_probabilities(traincsv, 'ethnicity', ['White-European','Asian', 'Middle Eastern ','South Asian','Latino','Pasifika','Hispanic','Turkish'], [0.45, 0.17, 0.15,0.10,0.05,0.05,0.02,0.01])
traincsv = fill_null_values_with_probabilities(traincsv, 'relation', ['self','Parent','Relative','Health care professional'], [0.76,0.19,0.04,0.01])

traincsv=traincsv.drop(['contry_of_res'], axis=1)

0


In [ ]:
traincsv=pd.get_dummies(traincsv, columns=['ethnicity', 'age_desc', 'relation'])

In [ ]:
traincsv.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,ethnicity_South Asian,ethnicity_Turkish,ethnicity_White-European,age_desc_'12-16 years',age_desc_'18 and more',age_desc_'4-11 years',relation_Health care professional,relation_Parent,relation_Relative,relation_self
0,1,0,1,0,1,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
len(traincsv.columns)

33

In [ ]:
traincsv['austim'].value_counts()

0    3140
1     556
Name: austim, dtype: int64

In [ ]:
traincsv['Class/ASD'].value_counts()

0    2651
1    1045
Name: Class/ASD, dtype: int64

In [ ]:
traincsv.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'jaundice', 'austim', 'used_app_before', 'result', 'Class/ASD',
       'ethnicity_Asian', 'ethnicity_Black', 'ethnicity_Hispanic',
       'ethnicity_Latino', 'ethnicity_Middle Eastern ', 'ethnicity_Pasifika',
       'ethnicity_South Asian', 'ethnicity_Turkish',
       'ethnicity_White-European', 'age_desc_'12-16 years'',
       'age_desc_'18 and more'', 'age_desc_'4-11 years'',
       'relation_Health care professional', 'relation_Parent',
       'relation_Relative', 'relation_self'],
      dtype='object')

In [ ]:
traincsv['Class/ASD']=traincsv['Class/ASD'].replace({'1':1, '0':0})
for col in traincsv.columns:
  if col not in ('age', 'result'):
    print(f'{col}: {traincsv[col].unique()}')

A1_Score: [1 0]
A2_Score: [0 1]
A3_Score: [1 0]
A4_Score: [0 1]
A5_Score: [1 0]
A6_Score: [0 1]
A7_Score: [1 0]
A8_Score: [0 1]
A9_Score: [1 0]
A10_Score: [1 0]
gender: [0 1]
jaundice: [0 1]
austim: [0 1]
used_app_before: [0 1]
Class/ASD: [0 1]
ethnicity_Asian: [0 1]
ethnicity_Black: [0 1]
ethnicity_Hispanic: [0 1]
ethnicity_Latino: [0 1]
ethnicity_Middle Eastern : [0 1]
ethnicity_Pasifika: [0 1]
ethnicity_South Asian: [0 1]
ethnicity_Turkish: [0 1]
ethnicity_White-European: [1 0]
age_desc_'12-16 years': [0 1]
age_desc_'18 and more': [1 0]
age_desc_'4-11 years': [0 1]
relation_Health care professional: [0 1]
relation_Parent: [0 1]
relation_Relative: [0 1]
relation_self: [1 0]


In [ ]:
min_max={}
for col in traincsv.columns:
  traincsv[col].astype(np.float64)
  min_max[col]=(traincsv[col].min(), traincsv[col].max())

In [ ]:
traincsv=(traincsv-traincsv.min())/(traincsv.max()-traincsv.min())
X=traincsv.drop(['Class/ASD'], axis=1)
y=traincsv['Class/ASD']

# Model Selection

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [ ]:
#ML models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

In [ ]:
models=[
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    DecisionTreeClassifier(),
    SVC(),
    MultinomialNB(),
    MLPClassifier(),
    KNeighborsClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    AdaBoostClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    RidgeClassifier(),
    SGDClassifier(),
    GaussianProcessClassifier()
]

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3696 entries, 0 to 3696
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   A1_Score                           3696 non-null   float64
 1   A2_Score                           3696 non-null   float64
 2   A3_Score                           3696 non-null   float64
 3   A4_Score                           3696 non-null   float64
 4   A5_Score                           3696 non-null   float64
 5   A6_Score                           3696 non-null   float64
 6   A7_Score                           3696 non-null   float64
 7   A8_Score                           3696 non-null   float64
 8   A9_Score                           3696 non-null   float64
 9   A10_Score                          3696 non-null   float64
 10  age                                3696 non-null   float64
 11  gender                             3696 non-null   float

In [ ]:
accuracies = []

for model in models:
    scores = cross_val_score(model, X, y, cv=5)  # 5-fold cross-validation
    accuracy = np.mean(scores)
    accuracies.append((model.__class__.__name__, accuracy))
    print(model.__class__.__name__, "done!!", accuracy)

RandomForestClassifier done!! 0.985388582086823
GradientBoostingClassifier done!! 0.9426533299199062
LogisticRegression done!! 0.8934092820831656
DecisionTreeClassifier done!! 0.972939326335808
SVC done!! 0.9158673152177889
MultinomialNB done!! 0.8246655451120946


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

MLPClassifier done!! 0.9293918004608127
KNeighborsClassifier done!! 0.915055407234027
GaussianNB done!! 0.8663493398676078
LinearDiscriminantAnalysis done!! 0.8714906923161321
QuadraticDiscriminantAnalysis done!! 0.8276505869875287


/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


AdaBoostClassifier done!! 0.9145243755257286
BaggingClassifier done!! 0.9742947006546465
ExtraTreesClassifier done!! 0.9824112204220459
RidgeClassifier done!! 0.873384047105292
SGDClassifier done!! 0.8774443184727353
GaussianProcessClassifier done!! 0.9204637384339686


In [ ]:
"""
RandomForestClassifier done!! 0.985388582086823
ExtraTreesClassifier done!! 0.9824112204220459
BaggingClassifier done!! 0.9742947006546465
DecisionTreeClassifier done!! 0.972939326335808
GradientBoostingClassifier done!! 0.9426533299199062

MLPClassifier done!! 0.9293918004608127
GaussianProcessClassifier done!! 0.9204637384339686
SVC done!! 0.9158673152177889
KNeighborsClassifier done!! 0.915055407234027
AdaBoostClassifier done!! 0.9145243755257286
LogisticRegression done!! 0.8934092820831656
SGDClassifier done!! 0.8774443184727353
RidgeClassifier done!! 0.873384047105292
LinearDiscriminantAnalysis done!! 0.8714906923161321
GaussianNB done!! 0.8663493398676078
QuadraticDiscriminantAnalysis done!! 0.8276505869875287
MultinomialNB done!! 0.8246655451120946

"""
print()

# Test data preprocessing

In [ ]:
testcsv=pd.read_csv("test_data_without_output.csv")
for col in traincsv.columns:
  if col not in ('age', 'result'):
    print(f'{col}: {traincsv[col].unique()}')

A1_Score: [1. 0.]
A2_Score: [0. 1.]
A3_Score: [1. 0.]
A4_Score: [0. 1.]
A5_Score: [1. 0.]
A6_Score: [0. 1.]
A7_Score: [1. 0.]
A8_Score: [0. 1.]
A9_Score: [1. 0.]
A10_Score: [1. 0.]
gender: [0. 1.]
jaundice: [0. 1.]
austim: [0. 1.]
used_app_before: [0. 1.]
Class/ASD: [0. 1.]
ethnicity_Asian: [0. 1.]
ethnicity_Black: [0. 1.]
ethnicity_Hispanic: [0. 1.]
ethnicity_Latino: [0. 1.]
ethnicity_Middle Eastern : [0. 1.]
ethnicity_Pasifika: [0. 1.]
ethnicity_South Asian: [0. 1.]
ethnicity_Turkish: [0. 1.]
ethnicity_White-European: [1. 0.]
age_desc_'12-16 years': [0. 1.]
age_desc_'18 and more': [1. 0.]
age_desc_'4-11 years': [0. 1.]
relation_Health care professional: [0. 1.]
relation_Parent: [0. 1.]
relation_Relative: [0. 1.]
relation_self: [1. 0.]


In [ ]:
testcsv['jaundice'] = testcsv['jaundice'].replace({"no":0, "yes":1})
testcsv['gender'] = testcsv['gender'].replace({"m":1, "f":0})
testcsv['used_app_before'] = testcsv['used_app_before'].replace({'no':0,"yes":1})
testcsv['relation'] = testcsv['relation'].replace({"Unkown":"others","Others":"others", "Self":"self", "'Health care professional'":"Health care professional", "?":"others"})
testcsv['ethnicity']=testcsv['ethnicity'].replace({"Others": "others", "?":"others", "'Middle Eastern '":"Middle Eastern ","Middle Eastern":"Middle Eastern ", "'South Asian'":"South Asian"})
testcsv['age_desc'] = testcsv['age_desc'].replace({"12-15 years":"12-16 years"})
testcsv['austim']=testcsv['austim'].replace({'yes':1, 'no':0})
testcsv['age']=testcsv['age'].replace({'?':None})
testcsv['age']=testcsv['age'].astype(np.float64)
testcsv=fill_na_with_column_average(testcsv, 'age')

for i in range(10):
  testcsv[f'A{i+1}_Score']=testcsv[f'A{i+1}_Score'].apply(lambda x: int(x))
results=testcsv[['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']].sum(axis=1)
testcsv['result']=results

testcsv['relation']=testcsv['relation'].replace({'others':None})
testcsv['ethnicity'] = testcsv['ethnicity'].replace({'others':None, "Unkown":None})
testcsv = fill_null_values_with_probabilities(testcsv, 'ethnicity', ['White-European','Asian', 'Middle Eastern ','South Asian','Latino','Pasifika','Hispanic','Turkish'], [0.45, 0.17, 0.15,0.10,0.05,0.05,0.02,0.01])
testcsv = fill_null_values_with_probabilities(testcsv, 'relation', ['self','Parent','Relative','Health care professional'], [0.76,0.19,0.04,0.01])

testcsv=testcsv.drop(['contry_of_res'], axis=1)

0


In [ ]:
testcsv=pd.get_dummies(testcsv, columns=['ethnicity', 'age_desc', 'relation'])

In [ ]:
testcsv.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'jaundice', 'austim', 'used_app_before', 'result', 'ethnicity_Asian',
       'ethnicity_Black', 'ethnicity_Hispanic', 'ethnicity_Latino',
       'ethnicity_Middle Eastern ', 'ethnicity_Pasifika',
       'ethnicity_South Asian', 'ethnicity_Turkish',
       'ethnicity_White-European', 'age_desc_12-16 years',
       'age_desc_18 and more', 'age_desc_4-11 years',
       'relation_Health care professional', 'relation_Parent',
       'relation_Relative', 'relation_self'],
      dtype='object')

In [ ]:
traincsv.columns

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age', 'gender',
       'jaundice', 'austim', 'used_app_before', 'result', 'Class/ASD',
       'ethnicity_Asian', 'ethnicity_Black', 'ethnicity_Hispanic',
       'ethnicity_Latino', 'ethnicity_Middle Eastern ', 'ethnicity_Pasifika',
       'ethnicity_South Asian', 'ethnicity_Turkish',
       'ethnicity_White-European', 'age_desc_'12-16 years'',
       'age_desc_'18 and more'', 'age_desc_'4-11 years'',
       'relation_Health care professional', 'relation_Parent',
       'relation_Relative', 'relation_self'],
      dtype='object')

In [ ]:
for col in testcsv.columns:
  if col not in ('age', 'result'):
    print(f'{col}: {testcsv[col].unique()}')

A1_Score: [1 0]
A2_Score: [1 0]
A3_Score: [1 0]
A4_Score: [1 0]
A5_Score: [1 0]
A6_Score: [0 1]
A7_Score: [1 0]
A8_Score: [1 0]
A9_Score: [1 0]
A10_Score: [1 0]
gender: [1 0]
jaundice: [1 0]
austim: [0 1]
used_app_before: [0 1]
ethnicity_Asian: [0 1]
ethnicity_Black: [0 1]
ethnicity_Hispanic: [0 1]
ethnicity_Latino: [0 1]
ethnicity_Middle Eastern : [1 0]
ethnicity_Pasifika: [0 1]
ethnicity_South Asian: [0 1]
ethnicity_Turkish: [0 1]
ethnicity_White-European: [0 1]
age_desc_12-16 years: [0 1]
age_desc_18 and more: [1 0]
age_desc_4-11 years: [0 1]
relation_Health care professional: [0 1]
relation_Parent: [0 1]
relation_Relative: [0 1]
relation_self: [1 0]


In [ ]:
testcsv=testcsv.rename(columns={"age_desc_12-16 years":"age_desc_'12-16 years'", "age_desc_18 and more":"age_desc_'18 and more'", "age_desc_4-11 years":"age_desc_'4-11 years'"})

In [ ]:
for col in testcsv.columns:
  testcsv[col]=(testcsv[col]-min_max[col][0])/(min_max[col][1]-min_max[col][0])

In [ ]:
for coltrain, coltest in zip(X.columns, testcsv.columns):
  if coltrain!=coltest:
    testcsv=testcsv.rename(columns={coltest:coltrain})

# Ensemble/Prediction

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
models=[
    RandomForestClassifier(),
    ExtraTreesClassifier(),
    BaggingClassifier(),
    DecisionTreeClassifier(),
    CatBoostClassifier(iterations=4000, eval_metric="Accuracy", verbose=0)
]

for model in models:
  model.fit(X, y)

predictions=pd.DataFrame()
final_preds=[]
for model in models:
  pred=model.predict(testcsv)
  final_preds.append(pred)
  predictions[model.__class__.__name__]=pred

In [ ]:
predictions

,RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,DecisionTreeClassifier,CatBoostClassifier
0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
574,0.0,0.0,0.0,0.0,0.0
575,0.0,0.0,0.0,0.0,0.0
576,0.0,0.0,0.0,0.0,0.0
577,1.0,1.0,1.0,1.0,1.0


In [ ]:
predictions.sum()

RandomForestClassifier    172.0
ExtraTreesClassifier      173.0
BaggingClassifier         174.0
DecisionTreeClassifier    168.0
CatBoostClassifier        175.0
dtype: float64

As Random Forest gave us a significantly higher accuracy than the other models previously, we decided to only use it and not use voting with each model

In [ ]:
sub_df=predictions[['RandomForestClassifier']].copy()
sub_df.columns

Index(['RandomForestClassifier'], dtype='object')

In [ ]:
sub_df=sub_df.rename(columns={"RandomForestClassifier":"Class/ASD"})

In [ ]:
sub_df.sum(axis=0)

Class/ASD    172.0
dtype: float64

In [ ]:
sub_df['Class/ASD']=sub_df['Class/ASD'].astype(np.int64)

In [ ]:
sub_df.sum()

Class/ASD    172
dtype: int64

In [ ]:
sub_df

,Class/ASD
0,1
1,0
2,1
3,0
4,0
...,...
574,0
575,0
576,0
577,1


In [ ]:
sub_df.to_csv("submission_final_random_forest.csv", index=False)